# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import h5py

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v1', file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)

### Env Name ###### svmEnv-v1
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


# Your codes `DDPG` and `PPO`

## Functions for saving and clean

In [3]:
## Save all rewards, energies and princip dims in files during training
def create_info_h5(agent, env):
    # Check if file exist and creat it
    i = 0
    while os.path.exists(f'runs_step_envs/run_{i}.hdf5'):
        i += 1
    dataFile = h5py.File(f'runs_step_envs/run_{i}.hdf5', 'a')
    
    # Create dataset to store info in hdf5 file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id}
    st = h5py.string_dtype(encoding='utf-8')
    dataFile.create_dataset('info', dtype=st)
    for k in info.keys():
        dataFile['info'].attrs[k] = info[k]

    # Create dataset to store hyperparams of the model in hdf5 file
    hyperparams = {'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
                   , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
                  , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
                   , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
                  , 'add_noise_every':agent.add_noise_every}
    dataFile.create_dataset('hyperparams', dtype='f')
    for k in hyperparams.keys():
        dataFile['hyperparams'].attrs[k] = hyperparams[k]
    
    # Create group for rewards, energies, princip dims, actor and critic model
    dataFile.create_group('sigmas')
    dataFile.create_group('rewards')
    dataFile.create_group('energies')
    dataFile.create_group('princip_dims')
    dataFile.create_group('actor_models')
    dataFile.create_group('critic_models')
    
    # Close and return data file name
    dataFile_name = dataFile.filename
    dataFile.close()
    
    return dataFile_name

def save_all(dat_file_name, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep \
             , act_model_i_ep, cr_model_i_ep):
    # Open data file
    dat_file = h5py.File(dat_file_name, 'a')
    
    # Create datasets for rewards, energies, pri dim and store data in it 
    dat_file['sigmas'].create_dataset(f'sigmas_ep_{i_ep}', dtype='f', data=sigmas_i_ep)
    dat_file['rewards'].create_dataset(f'rew_ep_{i_ep}', dtype='f', data=rew_i_ep)
    dat_file['energies'].create_dataset(f'en_ep_{i_ep}', dtype='f', data=en_i_ep)
    dat_file['princip_dims'].create_dataset(f'pri_dim_ep_{i_ep}', dtype='i', data=pri_dim_i_ep)
    
    # Store in actor models group the network params at each ep
    actor_model = torch.load(act_model_i_ep)
    dat_file['actor_models'].create_dataset(f'act_mod_{i_ep}', dtype='f')
    for k in actor_model.keys():
        dat_file['actor_models'][f'act_mod_{i_ep}'].attrs.create(name=k,data=actor_model[k].cpu().data.numpy())
    
    # Store in actor models group the network params at each ep
    critic_model = torch.load(cr_model_i_ep)
    dat_file['critic_models'].create_dataset(f'cri_mod_{i_ep}', dtype='f')
    for k in critic_model.keys():
        dat_file['critic_models'][f'cri_mod_{i_ep}'].attrs.create(name=k,data=critic_model[k].cpu().data.numpy())
    
    # Close data file
    dat_file.close()
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

## From my `ddpg_agent.py` code

In [4]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    dat_file_name = create_info_h5(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(dat_file_name=dat_file_name, i_ep=int(i_ep), sigmas_i_ep=env.actions_taken \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , act_model_i_ep='checkpoint_actor.pth', cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    rm_useless_file('checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return dat_file_name

In [6]:
all_data = run_ddpg(10, 10)

#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [60.13422  56.330334 59.012486]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0249627
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1964291659288975
#### CALL STEP #### 2
Action chosen at step:  [62.86827  45.008354 51.589306]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.024956
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1959632515227643
#### CALL STEP #### 3
Action chosen at step:  [61.541958 53.302563 48.138382]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.0130006
With this action the full dim is:  3  and princip dim is:  3
###

With this action the energy is:  0.0104117
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1845604310542743
#### CALL STEP #### 6
Action chosen at step:  [77.04065  57.397953 71.33675 ]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  0.0104057
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1841431942726608
#### CALL STEP #### 7
Action chosen at step:  [80.338425 54.125362 66.40992 ]
Basis size (it should be the same of full dim) =   7
With this action the energy is:  0.0103547
With this action the full dim is:  7  and princip dim is:  7
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1805966816289537
#### CALL STEP #### 8
Action chosen at step:  [74.71747  46.404324 61.930176]
Basis size (it should be the same of full dim) =   8
With this action th

With this action the energy is:  -0.0516462
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  3.1309126472204767
Episode 4 ... Score: 13.971
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [56.377586 52.58763  45.57889 ]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0272442
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.355083452137162
#### CALL STEP #### 2
Action chosen at step:  [56.56467  55.603527 47.160812]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0265087
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.3039371766577936
#### CALL ST

With this action the energy is:  -0.0420167
With this action the full dim is:  4  and princip dim is:  4
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.4612823824637458
#### CALL STEP #### 5
Action chosen at step:  [56.80586  11.017632 50.41968 ]
Basis size (it should be the same of full dim) =   5
With this action the energy is:  -0.0596782
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  3.6894536188728013
#### CALL STEP #### 6
Action chosen at step:  [45.94848  11.520611 58.228462]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  -0.0616659
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  3.827677210674759
#### CALL STEP #### 7
Action chosen at step:  [50.47058  11.630043 42.790047]
Basis size (it should be the same of full dim) =   7
With this action the

With this action the energy is:  -0.0246688
With this action the full dim is:  9  and princip dim is:  9
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.254918721840319
#### CALL STEP #### 10
Action chosen at step:  [30.34208  30.177835 35.993492]
Basis size (it should be the same of full dim) =   10
With this action the energy is:  -0.0247383
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.2597517145606663
Episode 9 ... Score: 2.895


## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break